In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)

In [2]:
def one_hot_encoder(X):
    N=len(X)
    K=len(set(X))
    Z=np.zeros((N,K))
    for i in range(N):
        c=X[i]
        Z[i,c]=1
    return Z
y_train=one_hot_encoder(y_train)

In [5]:
y_train.argmax(axis=1)

array([3, 6, 0, ..., 3, 6, 2], dtype=int64)

In [18]:
class LogiticSoftmax(object):
    def __init__(self,lr,epoch):
        self.lr=lr
        self.epoch=epoch
    def forward(self,X,W,b):
        return self.softmax(X.dot(W)+b)
    def softmax(self,a):
        return np.exp(a)/(np.exp(a).sum(axis=1,keepdims=True))
    def cost(self,y,y_p):
        return -(y*np.log(y_p)).sum()
    def fit(self,X,y):
        N,D=X.shape
        K=y.shape[1]
        W=np.random.randn(D,K)
        b=np.random.randn(K)
        tr_i=np.random.choice(N,int(N*0.9),replace=False)
        t_i=[i for i in range(N) if i not in tr_i]
        X_t=X[tr_i]
        y_t=y[tr_i]
        X_v=X[t_i]
        y_v=y[t_i]
        c_t=[]
        c_v=[]
        cl_t=[]
        cl_v=[]
        for i in range(self.epoch):
            y_p_t=self.forward(X_t,W,b)
            y_p_v=self.forward(X_v,W,b)
            dy=y_p_t-y_t
            W=W-self.lr*(X_t.T.dot(dy))
            b=b-self.lr*(dy).sum(axis=0)
            c_t.append(self.cost(y_t,y_p_t))
            c_v.append(self.cost(y_v,y_p_v))
            cl_t.append(self.score(y_t.argmax(axis=1),y_p_t.argmax(axis=1)))
            cl_v.append(self.score(y_v.argmax(axis=1),y_p_v.argmax(axis=1)))
            if i % 100==0:
                print('Epoch {0} Train Cost: {1} , Classification: {2} , Test Cost {3} , Classification : {4}'.format(i,c_t[i],
                                                                                                                      cl_t[i],
                                                                                                                      c_v[i],
                                                                                                                      cl_v[i]))
        self.W=W
        self.b=b
        if fig==True:
            plt.plot(c_t,label='Train Cost')
            plt.plot(c_v,label='Test Cost')
            plt.show()
            plt.plot(cl_t,label='Train Class')
            plt.plot(cl_v,label='Test Class')
            plt.show()
    def predict(self,X):
        return self.forward(X,self.W,self.b).argmax(axis=1)
    def score(self,y,y_p):
        return np.mean(y==y_p)

In [13]:
logit=LogiticSoftmax(0.000001,10000)

In [14]:
logit.fit(X_train,y_train)

Epoch 0 Train Cost: 662241.690763412 , Classification: 0.11229595886392656 , Test Cost 71707.22094875021 , Classification : 0.10590062111801242
Epoch 100 Train Cost: 310007.1012596489 , Classification: 0.14490803050695378 , Test Cost 34382.106885665984 , Classification : 0.14440993788819875
Epoch 200 Train Cost: 272987.2622191332 , Classification: 0.15156848535045037 , Test Cost 30283.840990369066 , Classification : 0.14472049689440994
Epoch 300 Train Cost: 250489.72777024482 , Classification: 0.15729716671843186 , Test Cost 27811.962322527703 , Classification : 0.15031055900621118
Epoch 400 Train Cost: 234159.83675646823 , Classification: 0.16509645580978016 , Test Cost 26012.81474161896 , Classification : 0.15527950310559005
Epoch 500 Train Cost: 221535.81853356306 , Classification: 0.1728612347724057 , Test Cost 24618.583219639215 , Classification : 0.16149068322981366
Epoch 600 Train Cost: 211405.52016300007 , Classification: 0.17821030472443664 , Test Cost 23503.436409048692 , Cla

Epoch 5600 Train Cost: 111787.14885026995 , Classification: 0.2571349691134348 , Test Cost 13165.841480266725 , Classification : 0.24006211180124223
Epoch 5700 Train Cost: 111164.1353959269 , Classification: 0.2569624184698209 , Test Cost 13101.408177794308 , Classification : 0.2406832298136646
Epoch 5800 Train Cost: 110554.61075781536 , Classification: 0.25779066155916763 , Test Cost 13038.268181045327 , Classification : 0.2422360248447205
Epoch 5900 Train Cost: 109958.08643582255 , Classification: 0.2582738033612865 , Test Cost 12976.373829180384 , Classification : 0.24192546583850932
Epoch 6000 Train Cost: 109374.09809631268 , Classification: 0.2590675363219105 , Test Cost 12915.680168557941 , Classification : 0.2422360248447205
Epoch 6100 Train Cost: 108802.20403078373 , Classification: 0.25982675915381165 , Test Cost 12856.144755828287 , Classification : 0.24161490683229814
Epoch 6200 Train Cost: 108241.98377123913 , Classification: 0.26034441108465334 , Test Cost 12797.7274763272

NameError: name 'fig' is not defined

In [15]:
y_test

array([0, 1, 4, ..., 0, 3, 2], dtype=int64)

In [19]:
logit.predict(X_test)

NameError: name 'forward' is not defined

In [22]:
logit.W.shape

(2304, 7)